# Baseline RAG テスト

こちらを参考にBaseline RAGを構築TEST：

https://note.com/noa813/n/n6065209e7bf0

### ライブラリーのインストール
!pip install langchain openai chromadb tiktoken

### OpenAIのAPIキーを環境変数に格納

In [2]:
import openai
import os
import getpass

# !初回のみ! openaiのAPIキーを環境変数として設定

# OpenAI APIのアカウントを作成、
# https://platform.openai.com/account/api-keys
# からAPIキーを取得して環境変数に設定

# os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI_API_KEY: ")

# openai.api_key = os.getenv("OPENAI_API_KEY")

### LLMが参照するテキストを用意

In [3]:
file_path = './data/manufacturing_company_dummy_v2.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

text


'名前: 田中 太郎\n所属部門: 生産部\n所属部署: 第1製造課\n役職: 課長\n企業での経歴の詳細:\n田中太郎は、生産部第1製造課の課長として、生産ラインの管理を担当しています。彼は「A工程ランニングコスト削減プロジェクト」、「生産ライン自動化導入プロジェクト」、「D工程作業手順書改訂プロジェクト」に参加し、生産効率の向上に大きく貢献しました。同僚の佐藤次郎と協力して、生産プロセスの最適化を進めています。彼は製造業務管理士の資格を持ち、Lean生産方式に精通しています。\n\n名前: 鈴木 花子\n所属部門: 品質管理部\n所属部署: 品質保証課\n役職: 主任\n企業での経歴の詳細:\n鈴木花子は、品質保証課の主任として、製品の品質検査を統括しています。「品質管理システム改善プロジェクト」、「品質認証取得プロジェクト」、「環境負荷低減プロジェクト」に参加し、品質基準の向上と環境対応に努めています。彼女は高橋一郎と協力し、品質データの分析を行っています。品質管理検定1級の資格を持ち、ISO 9001の知識があります。\n\n名前: 佐藤 次郎\n所属部門: 生産部\n所属部署: 第2製造課\n役職: 係長\n企業での経歴の詳細:\n佐藤次郎は、第2製造課の係長で、生産ラインの運用を担当しています。「C工程生産性向上プロジェクト」、「在庫管理効率化プロジェクト」、「E工程レイアウト変更プロジェクト」に参加し、生産性と在庫管理の改善に寄与しました。田中太郎と共同で作業手順の最適化を進めています。フォークリフト運転技能者の資格を持ち、在庫管理システムの知識があります。\n\n名前: 高橋 一郎\n所属部門: 技術開発部\n所属部署: 開発課\n役職: マネージャー\n企業での経歴の詳細:\n高橋一郎は、開発課のマネージャーとして新製品の開発をリードしています。「新製品開発プロジェクト」、「ITシステム統合プロジェクト」、「マーケティング戦略策定プロジェクト」に参加し、製品開発と市場戦略の立案に貢献しました。彼は鈴木花子と協力して品質基準の設定を行っています。プロジェクトマネージャー資格を持ち、機械工学の専門知識があります。\n\n名前: 伊藤 美咲\n所属部門: 人事部\n所属部署: 人材開発課\n役職: スペシャリスト\n企業での経歴の詳細:\n伊藤美咲は、人

### テキストを分割

In [4]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="。\n\n",
    chunk_size=200, 
    chunk_overlap=0
    )

docs = text_splitter.split_text(text)

print(f"{len(docs)} chunks")
docs

Created a chunk of size 235, which is longer than the specified 200
Created a chunk of size 218, which is longer than the specified 200
Created a chunk of size 217, which is longer than the specified 200
Created a chunk of size 222, which is longer than the specified 200
Created a chunk of size 227, which is longer than the specified 200
Created a chunk of size 208, which is longer than the specified 200
Created a chunk of size 222, which is longer than the specified 200
Created a chunk of size 212, which is longer than the specified 200
Created a chunk of size 214, which is longer than the specified 200
Created a chunk of size 218, which is longer than the specified 200
Created a chunk of size 211, which is longer than the specified 200
Created a chunk of size 223, which is longer than the specified 200
Created a chunk of size 214, which is longer than the specified 200
Created a chunk of size 216, which is longer than the specified 200
Created a chunk of size 207, which is longer tha

50 chunks


['名前: 田中 太郎\n所属部門: 生産部\n所属部署: 第1製造課\n役職: 課長\n企業での経歴の詳細:\n田中太郎は、生産部第1製造課の課長として、生産ラインの管理を担当しています。彼は「A工程ランニングコスト削減プロジェクト」、「生産ライン自動化導入プロジェクト」、「D工程作業手順書改訂プロジェクト」に参加し、生産効率の向上に大きく貢献しました。同僚の佐藤次郎と協力して、生産プロセスの最適化を進めています。彼は製造業務管理士の資格を持ち、Lean生産方式に精通しています',
 '名前: 鈴木 花子\n所属部門: 品質管理部\n所属部署: 品質保証課\n役職: 主任\n企業での経歴の詳細:\n鈴木花子は、品質保証課の主任として、製品の品質検査を統括しています。「品質管理システム改善プロジェクト」、「品質認証取得プロジェクト」、「環境負荷低減プロジェクト」に参加し、品質基準の向上と環境対応に努めています。彼女は高橋一郎と協力し、品質データの分析を行っています。品質管理検定1級の資格を持ち、ISO 9001の知識があります',
 '名前: 佐藤 次郎\n所属部門: 生産部\n所属部署: 第2製造課\n役職: 係長\n企業での経歴の詳細:\n佐藤次郎は、第2製造課の係長で、生産ラインの運用を担当しています。「C工程生産性向上プロジェクト」、「在庫管理効率化プロジェクト」、「E工程レイアウト変更プロジェクト」に参加し、生産性と在庫管理の改善に寄与しました。田中太郎と共同で作業手順の最適化を進めています。フォークリフト運転技能者の資格を持ち、在庫管理システムの知識があります',
 '名前: 高橋 一郎\n所属部門: 技術開発部\n所属部署: 開発課\n役職: マネージャー\n企業での経歴の詳細:\n高橋一郎は、開発課のマネージャーとして新製品の開発をリードしています。「新製品開発プロジェクト」、「ITシステム統合プロジェクト」、「マーケティング戦略策定プロジェクト」に参加し、製品開発と市場戦略の立案に貢献しました。彼は鈴木花子と協力して品質基準の設定を行っています。プロジェクトマネージャー資格を持ち、機械工学の専門知識があります',
 '名前: 伊藤 美咲\n所属部門: 人事部\n所属部署: 人材開発課\n役職: スペシャリスト\n企業での経歴の詳細:\n伊藤美咲は、

### OpenAIのEmbeddingのインスタンス化

* チャンクのベクトル化を行っている
* 追加インストールが必要：pip install -U langchain-community

In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

C:\Users\user\AppData\Local\Temp\ipykernel_14384\1845447191.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


### VectorStore作成
これがベクトル化されたチャンクを保存するためのベクトルDB

In [6]:
from langchain.vectorstores import Chroma
db = Chroma.from_texts(docs, embeddings)
retriever = db.as_retriever()

### 質問（Query）を用意

質問（query）がベクトル化され、
queryと類似度が近いベクトルを持つ
テキストをVectorStoreから検索します。
そして、
queryに関連する文書が
context_docsへ格納されます

In [12]:
# query = "人的資本の強化のために取り組んでいることは何でしょうか？"
# query = "田中太郎と一緒にプロジェクトに従事したことがある社員を全員挙げてください"
# query = "この企業内で生産ラインのDXプロジェクトを企画することになりました。マネージャークラスから担当者クラスまでの社員5名を当プロジェクトのメンバーとして選定してください。"
query = "環境負荷低減プロジェクトに関わった社員全員を挙げてください。"

context_docs = retriever.get_relevant_documents(query)
context_docs

[Document(metadata={}, page_content='名前: 野村 一郎\n所属部門: 品質管理部\n所属部署: 環境管理課\n役職: マネージャー\n企業での経歴の詳細:\n野村一郎は、環境管理課のマネージャーとして環境対策を統括しています。「環境負荷低減プロジェクト」、「品質認証取得プロジェクト」、「サプライチェーン最適化プロジェクト」に参加し、環境負荷の低減に貢献しました。佐々木玲と協力して、環境マネジメントシステムの構築を進めています。環境管理士の資格を持ち、環境法規制に詳しいです'),
 Document(metadata={}, page_content='名前: 福田 美紀\n所属部門: 人事部\n所属部署: 福利厚生課\n役職: スペシャリスト\n企業での経歴の詳細:\n福田美紀は、福利厚生課のスペシャリストとして社員の福利厚生を担当しています。「社員研修プログラム見直しプロジェクト」、「安全管理体制強化プロジェクト」、「環境負荷低減プロジェクト」に参加し、働きやすい職場環境づくりに努めています。伊藤美咲と協力して、社員満足度の向上を目指しています。社会保険労務士の資格を持ち、労務管理に詳しいです'),
 Document(metadata={}, page_content='名前: 宮崎 大輔\n所属部門: 技術開発部\n所属部署: 研究開発課\n役職: リサーチャー\n企業での経歴の詳細:\n宮崎大輔は、研究開発課のリサーチャーとして新技術の研究を行っています。「新製品開発プロジェクト」、「環境負荷低減プロジェクト」、「ITシステム統合プロジェクト」に参加し、技術革新に貢献しました。高橋一郎と協力して、研究成果の製品化を進めています。博士号を持ち、材料工学に精通しています'),
 Document(metadata={}, page_content='名前: 佐々木 玲\n所属部門: 購買部\n所属部署: 物流管理課\n役職: マネージャー\n企業での経歴の詳細:\n佐々木玲は、物流管理課のマネージャーとして物流全般を統括しています。「在庫管理効率化プロジェクト」、「サプライチェーン最適化プロジェクト」、「環境負荷低減プロジェクト」に参加し、物流コストの削減と環境対策に努めています。井上剛と協力して、物流プロセスの改善を行って

### *** LLMに質問を投げ、RAGを使って回答

In [13]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0
)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)

result = qa_chain.run(query)
print(result)

環境負荷低減プロジェクトに関わった社員は以下の通りです：

1. 野村 一郎
2. 福田 美紀
3. 宮崎 大輔
4. 佐々木 玲


### ChatGPT　モデル間の回答比較

##### ダミーデータ(製造業企業の社員情報 v1) で検証
**Query #1:「山田太郎はどのような人物ですか？」**

*retriever結果(どのチャンクが回答根拠として選ばれたか？)


[Document(metadata={}, page_content='名前： 山田太郎\n所属部門： 製造\n所属部署： 生産統括\n役職： 部長\n経歴詳細：\n本企業で20年以上にわたり製造ラインにおけるA工程の管理・改善に関わった経験を持つ。佐藤健太とは10年前にB工程ラインで共にライン管理に従事し、B工程に関する知識も豊富である。今まで関わったプロジェクトとしては以下がある：\nA工程ランニングコスト削減プロジェクト\nB工程製造設備オーバーホールプロジェクト\n新卒人材採用活動'),
 
 
 Document(metadata={}, page_content='名前： 上田健太\n所属部門： 研究開発\n所属部署： 技術開発\n役職： エンジニア\n経歴詳細：\n原田涼と新製品開発プロジェクトに参加。環境負荷低減プロジェクトでも技術的貢献。関わったプロジェクト：\n新製品開発プロジェクト\n環境負荷低減プロジェクト\n生産ライン自動化導入プロジェクト'),
 
 
 Document(metadata={}, page_content='名前： 谷口健太\n所属部門： 研究開発\n所属部署： 技術開発\n役職： エンジニア\n経歴詳細：\n上田健太と新製品開発プロジェクトに参加。環境負荷低減プロジェクトでも技術的貢献。関わったプロジェクト：\n新製品開発プロジェクト\n環境負荷低減プロジェクト\n生産ライン自動化導入プロジェクト'),
 
 
 Document(metadata={}, page_content='名前： 橋本健太郎\n所属部門： 製造\n所属部署： 生産管理\n役職： 課長\n経歴詳細：\n生産ラインの効率化に注力。木村拓也と在庫管理効率化プロジェクトで協力。参加したプロジェクト：\n在庫管理効率化プロジェクト\nコスト削減アイデア募集活動\n業務プロセス改善プロジェクト')]



**LLMの回答**

* turbo-3.5
「山田太郎は製造部門に所属し、生産統括部署の部長です。20年以上の経験を持ち、A工程の管理・改善に関わってきました。10年前にはB工程ラインでライン管理に従事し、B工程にも豊富な知識を持っています。過去に関わったプロジェクトにはA工程ランニングコスト削減プロジェクト、B工程製造設備オーバーホールプロジェクト、新卒人材採用活動などがあります。」

* 4o

「山田太郎は、製造部門の生産統括部署に所属する部長です。本企業で20年以上にわたり製造ラインにおけるA工程の管理・改善に関わった経験を持ち、B工程に関する知識も豊富です。彼はA工程ランニングコスト削減プロジェクトやB工程製造設備オーバーホールプロジェクト、新卒人材採用活動などに関わってきました。また、佐藤健太とは10年前にB工程ラインで共にライン管理に従事した経験があります。」

------

##### ダミーデータ(製造業企業の社員情報 v2) で検証
**Query #1:「この企業内で生産ラインのDXプロジェクトを企画することになりました。マネージャークラスから担当者クラスまでの社員5名を当プロジェクトのメンバーとして選定してください。」**

**LLMの回答**

* turbo-3.5

「### ChatGPT　モデル間の回答比較

##### ダミーデータ(製造業企業の社員情報 v1) で検証
**Query #1:「山田太郎はどのような人物ですか？」**

*retriever結果(どのチャンクが回答根拠として選ばれたか？)


[Document(metadata={}, page_content='名前： 山田太郎\n所属部門： 製造\n所属部署： 生産統括\n役職： 部長\n経歴詳細：\n本企業で20年以上にわたり製造ラインにおけるA工程の管理・改善に関わった経験を持つ。佐藤健太とは10年前にB工程ラインで共にライン管理に従事し、B工程に関する知識も豊富である。今まで関わったプロジェクトとしては以下がある：\nA工程ランニングコスト削減プロジェクト\nB工程製造設備オーバーホールプロジェクト\n新卒人材採用活動'),
 
 
 Document(metadata={}, page_content='名前： 上田健太\n所属部門： 研究開発\n所属部署： 技術開発\n役職： エンジニア\n経歴詳細：\n原田涼と新製品開発プロジェクトに参加。環境負荷低減プロジェクトでも技術的貢献。関わったプロジェクト：\n新製品開発プロジェクト\n環境負荷低減プロジェクト\n生産ライン自動化導入プロジェクト'),
 
 
 Document(metadata={}, page_content='名前： 谷口健太\n所属部門： 研究開発\n所属部署： 技術開発\n役職： エンジニア\n経歴詳細：\n上田健太と新製品開発プロジェクトに参加。環境負荷低減プロジェクトでも技術的貢献。関わったプロジェクト：\n新製品開発プロジェクト\n環境負荷低減プロジェクト\n生産ライン自動化導入プロジェクト'),
 
 
 Document(metadata={}, page_content='名前： 橋本健太郎\n所属部門： 製造\n所属部署： 生産管理\n役職： 課長\n経歴詳細：\n生産ラインの効率化に注力。木村拓也と在庫管理効率化プロジェクトで協力。参加したプロジェクト：\n在庫管理効率化プロジェクト\nコスト削減アイデア募集活動\n業務プロセス改善プロジェクト')]



**LLMの回答**

* turbo-3.5
「山田太郎は製造部門に所属し、生産統括部署の部長です。20年以上の経験を持ち、A工程の管理・改善に関わってきました。10年前にはB工程ラインでライン管理に従事し、B工程にも豊富な知識を持っています。過去に関わったプロジェクトにはA工程ランニングコスト削減プロジェクト、B工程製造設備オーバーホールプロジェクト、新卒人材採用活動などがあります。」

* 4o

「山田太郎は、製造部門の生産統括部署に所属する部長です。本企業で20年以上にわたり製造ラインにおけるA工程の管理・改善に関わった経験を持ち、B工程に関する知識も豊富です。彼はA工程ランニングコスト削減プロジェクトやB工程製造設備オーバーホールプロジェクト、新卒人材採用活動などに関わってきました。また、佐藤健太とは10年前にB工程ラインで共にライン管理に従事した経験があります。」

------

##### ダミーデータ(製造業企業の社員情報 v2) で検証
**Query #1:「この企業内で生産ラインのDXプロジェクトを企画することになりました。マネージャークラスから担当者クラスまでの社員5名を当プロジェクトのメンバーとして選定してください。」**

**LLMの回答**

* turbo-3.5

「生産ラインのDXプロジェクトのメンバーとして以下の5名を選定できます:

1. 田中太郎 - 生産部第1製造課の課長で、生産ラインの管理を担当しており、Lean生産方式に精通しています。
2. 高橋一郎 - 技術開発部のマネージャーで、新製品開発や市場戦略の立案に貢献しています。機械工学の専門知識があります。
3. 藤田直人 - 技術開発部のプロジェクト管理課のプロジェクトマネージャーで、各種プロジェクトを統括しています。PMP資格を持ち、プロジェクト管理に精通しています。
4. 佐藤次郎 - 生産部第2製造課の係長で、生産ラインの運用を担当しています。在庫管理システムの知識があります。
5. 鈴木花子 - 高橋一郎と協力して品質基準の設定を行っている可能性があるため、品質管理の観点からプロジェクトに貢献できるかもしれません。

これらのメンバーは、それぞれの専門知識や経験を活かして生産ラインのDXプロジェクトに貢献できる可能性があります。」


* 4o


「生産ラインのDXプロジェクトに適したメンバーとして、以下の5名を選定することが考えられます：

1. 田中 太郎 - 生産部第1製造課の課長で、生産ラインの管理に精通しており、過去に生産ライン自動化導入プロジェクトに参加した経験があります。

2. 高橋 一郎 - 技術開発部の開発課マネージャーで、新製品開発やITシステム統合プロジェクトに参加した経験があり、技術的な知識を持っています。

3. 藤田 直人 - 技術開発部のプロジェクト管理課のプロジェクトマネージャーで、プロジェクト管理に精通しており、プロジェクトの進捗管理を行う能力があります。

4. 佐藤 次郎 - 生産部第2製造課の係長で、生産ラインの運用に関する知識があり、田中太郎と共同で作業手順の最適化を進めた経験があります。

5. 鈴木 花子 - 高橋一郎と協力して品質基準の設定を行っているため、品質管理の観点からプロジェクトに貢献できる可能性があります。（鈴木花子の詳細は提供されていませんが、品質基準の設定に関与していることから選定しました）

このメンバー構成により、プロジェクトは生産ラインの技術的な側面、管理面、品質管理の観点からバランスよく進められるでしょう。
」


**Query #2:「」**



**LLMの回答**

* turbo-3.5


* 4o


**Query #3:「」**



**LLMの回答**

* turbo-3.5


* 4o



